In [1]:
from llama_index.core.postprocessor import (PIINodePostprocessor, NERPIINodePostprocessor)
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.schema import TextNode

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from llama_parse import LlamaParse
import os
import dotenv

dotenv.load_dotenv()

True

In [4]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

embed_model = OllamaEmbedding(model_name="nomic-embed-text")
ollama_llm = Ollama(model="phi3", request_timeout=60.0, temperature=0)

In [5]:
text = """
My name is John Doe and I live in New York. My name is Jane Doe and I live in California.
My full address is 123 Main St, New York, NY 10001. I work at XYZ Corp. My email is 5iSjB@example.com
"""

In [6]:
node = TextNode(text=text)
node

TextNode(id_='04226cd2-049e-41fb-abcc-fcbd6194fab1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\nMy name is John Doe and I live in New York. My name is Jane Doe and I live in California.\nMy full address is 123 Main St, New York, NY 10001. I work at XYZ Corp. My email is 5iSjB@example.com\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [7]:
processor = PIINodePostprocessor(llm=ollama_llm)

In [8]:
from llama_index.core.schema import NodeWithScore

new_nodes = processor.postprocess_nodes([NodeWithScore(node=node)])
new_nodes[0].node.get_text()

'Hello [NAME1], I am [NAME2]. My full address is at [ADDRESS] working at XYZ Corp. My email is [EMAIL_ID].'

In [9]:
new_nodes[0].node.metadata["__pii_node_info__"]

{'NAME1': 'John Doe',
 'NAME2': 'Jane Doe',
 'ADDRESS': '[MASKED] Street, California, CA 56005',
 'EMAIL_ID': '[MASKED]@example.com'}